In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
def store_relational_JH_data():
    ''' Transformes the COVID data in a relational data set
    '''
data_path1='file:///C:/Users/navid/covid/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path1)
pdb=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})
pdb['state']=pdb['state'].fillna('no')
pdb=pdb.drop(['Lat','Long'],axis=1)
prm=pdb.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
prm['date']=prm.date.astype('datetime64[ns]')
prm.to_csv('C:/Users/navid/covid/data/processed/COVID_relational_confirmed.csv',sep=';',index=False)

if __name__ == '__main__':

    store_relational_JH_data()

Navid Azadegan 

In [2]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from datetime import datetime
pd.options.display.max_rows=8 #pd.set_option('display.max_rows', 500 )
%matplotlib inline


Data Gathering

In [3]:
data_path1='file:///C:/Users/navid/covid/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
data_path2='file:///C:/Users/navid/covid/data/raw/COVID-19/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv'
data_path3='file:///C:/Users/navid/covid/data/raw2/COVID-19/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv'
pd_raw1=pd.read_csv(data_path1) #covid cases
pd_raw2=pd.read_csv(data_path2) #Country Population
pd_raw3=pd.read_csv(data_path3) #Vaccine data


In [4]:
pd_raw2

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.939110,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.153300,20.168300,Albania,2877800.0
2,10,AQ,ATA,10.0,NaN,NaN,NaN,Antarctica,-71.949900,23.347000,Antarctica,NaN
3,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.033900,1.659600,Algeria,43851043.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4313,84056039,US,USA,840.0,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",23464.0
4314,84056041,US,USA,840.0,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",20226.0
4315,84056043,US,USA,840.0,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",7805.0
4316,84056045,US,USA,840.0,56045.0,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",6927.0


In [5]:
#type(pd_raw2)
df = pd.DataFrame(pd_raw2)
type(df)

pandas.core.frame.DataFrame

In [6]:
df['Province_State'].isnull()
df['Province_State'].notnull() 

0       False
1       False
2       False
3       False
        ...  
4313     True
4314     True
4315     True
4316     True
Name: Province_State, Length: 4317, dtype: bool

Data Preparation & Arragging

In [7]:
pd_data_base=pd_raw3.rename(columns={'Country_Region':'country','People_fully_vaccinated':'vaccin','Date':'date'})
pd_data_base=pd_data_base.drop(['Province_State','UID', 'Report_Date_String','Doses_admin','People_partially_vaccinated'], axis=1)
pd_data_base=pd_data_base.dropna()
pd_data_base['date']=pd_data_base.date.astype('datetime64[ns]')


In [8]:
table = pd.pivot_table(pd_data_base, values='vaccin', index=['country'],
                    columns=['date'], aggfunc=np.sum)

In [9]:
pd_raw3=table.reset_index()

#pd_raw3

In [10]:
Time_indx=pd_raw1.columns[4:]
Time_indx3=pd_raw3.columns[2:]

In [11]:
df_plot=pd.DataFrame({'date':Time_indx})
df_plot3=pd.DataFrame({'date':Time_indx3})


In [12]:
pd_raw2.Population=pd_raw2.Population.fillna(1)
Country_list=pd_raw2.Country_Region.unique()



------------------------------------------------------Ploting and Checking Data--------------------------------------------- 

In [ ]:
for each in Country_list:
    Country_Population=max(pd_raw2[pd_raw2['Country_Region']==each].Population)
    df_plot[each]=np.array(pd_raw1[pd_raw1['Country/Region']==each].iloc[:,4::].sum(axis=0)/Country_Population)
#df_plot.head()

In [14]:
df_plot['date']=df_plot.date.astype('datetime64[ns]')

"Relative cases overtime of Covid infectors" first view

In [15]:

#df_plot.set_index('date').plot()

In [ ]:
for each in Country_list:
    Country_Population=max(pd_raw2[pd_raw2['Country_Region']==each].Population)
    df_plot3[each]=np.array(pd_raw3[pd_raw3['country']==each].iloc[:,2::].sum(axis=0)/Country_Population)
#df_plot3.head()  

In [17]:
df_plot3['date']=df_plot3.date.astype('datetime64[ns]')

"Vaccination rate (percentage of the population) over time" first view

In [18]:
#df_plot3.set_index('date').plot()

-------------------------------------------------------------------------------Saving Data--------------------------------------------

In [19]:
data_path4='C:/Users/navid/covid/data/processed/saved_covid.csv'#processed raw1
pd_raw3.to_csv(data_path4, index= False)

------------------------------------------------------------------------------Ploting (5.2)

In [20]:
import dash
from dash import dcc
from dash import html
fig=go.Figure()
#import dash_core_components as dcc
#import dash_html_components as html

app = dash.Dash()
app.layout = html.Div([
    html.Label('Multi-Select Country'),
    dcc.Dropdown(
        id='country_drop_down',
        options=[{'label': each, 'value': each} for each in Country_list],
                    
        value=['US', 'Germany'],
        multi=True
    ),


    dcc.Graph(figure=fig, id='main_window_slope')
])

In [21]:
from dash.dependencies import Input, Output

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country_list):

    traces = []
    for each in country_list:
        traces.append(dict(x=df_plot.date,
                           y=df_plot[each],
                          mode='markers+lines',
                          opacity=0.9,
                           line_width=2,
                          marker_size=4,
                          name=each))



    return {
            'data': traces,
            'layout':dict (
                     width=1200,
                     height=700
                     , xaxis_title="Time"
                     , yaxis_title="Vaccin Dose (Source: John Hopkins csse)")
                     
                       
                               
           
     }
# title="--- (Source: John Hopkins csse) monitored by Navid Azadegan (Mtr : 421820) ---"

In [ ]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
